<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/JGroups-master/jgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=859b5302836eccf8ca92e9f0544f9a844bcb9aea98d278252d1987233835b3a0
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [2]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [3]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

save_file_csv = 'JGroups2.csv'
file_csv =  'JGroups.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master'

In [6]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [7]:
file_name(file_dir)

In [8]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/DefaultSocketFactory.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/NAMING.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/MergeId.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/MsgStats.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/AwaitInfo.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/Channel.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/Frag3Header.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/GridOutputStream.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/MethodLookup.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/PaddedAtomicLong.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/SocketFactory.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/MFC.txt',
 '/content/drive/MyD

In [9]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [10]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line)
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese)
    #print("unchinese:",unchinese)
    return unchinese

In [11]:
def del_stopwords(line):
#     for line in file.readlines():
#     dicts = {i: '' for i in punctuation}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

#     for word in stopwords:
#           new_line = new_line.replace(word, '')

    return new_line


In [12]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
#             if len(coefs) != 0:
#                 if coefs != ' ':
#                     if coefs != '\n':
            for i in coefs:
                list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)
            


T
T
T
T
T
T
T
T
T
T
T
T
T


In [13]:
for k in range(0,len(Brain_Class)):
    a = Brain_Class[k][0]
    if '_' in a:
        n = a.find('_')
        new_str = ''
        for i in range(0, n):
            new_str = new_str + a[i]
        Brain_Class[k][0] = new_str
    print(Brain_Class[k][0])

DefaultSocketFactory
NAMING
MergeId
MsgStats
AwaitInfo
Channel
Frag3Header
GridOutputStream
MethodLookup
PaddedAtomicLong
SocketFactory
MFC
NonReflectiveProbeHandler
MethodInvoker
OneTimeAddressGenerator
SiteAddress
ExecutorEvent
GridInputStream
ExecutionServiceDemo
FutureListener
MergeData
Global
RequestHandler
DeliveryManager
DeltaView
Buffer
DH
SIZE
ByteArrayDataInputStream
RspFilter
Condition
MessageID
MergeView
MembershipChangePolicy
PERF
NonBlockingCredit
MessageDialog
FD
PropertiesToAsciidoc
AnycastAddress
MULTI
AsyncNoBundler
NoProgressException
ExecutionRunner
ConcurrentLinkedBlockingQueue
Average
CounterService
FileObserver
AgeOutCache
LockInfo
LockServiceDemo
ImmutableReference
DeliveryProtocol
UFC
FcHeader
LongTuple
ForkProtocol
ProtocolHook
SERIALIZE
DISCARD
Log4J2LogImpl
OutputStreamAdapter
XMLSchemaGenerator
SWIFT
Credit
LockService
ABP
DELIVERY
ConfiguratorFactory
NioServer
PaddedAtomicInteger
ArrayIterator
FragHeader
Marshaller
KeyStoreGenerator
INJECT
Base64
UnknownFo

In [14]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['DefaultSocketFactory', 'False'],
       ['NAMING', 'False'],
       ['MergeId', 'False'],
       ['MsgStats', 'False'],
       ['AwaitInfo', 'False'],
       ['Channel', 'False'],
       ['Frag3Header', 'False'],
       ['GridOutputStream', 'False'],
       ['MethodLookup', 'False'],
       ['PaddedAtomicLong', 'False'],
       ['SocketFactory', 'False'],
       ['MFC', 'False'],
       ['NonReflectiveProbeHandler', 'False'],
       ['MethodInvoker', 'False'],
       ['OneTimeAddressGenerator', 'False'],
       ['SiteAddress', 'False'],
       ['ExecutorEvent', 'False'],
       ['GridInputStream', 'False'],
       ['ExecutionServiceDemo', 'False'],
       ['FutureListener', 'False'],
       ['MergeData', 'False'],
       ['Global', 'False'],
       ['RequestHandler', 'False'],
       ['DeliveryManager', 'False'],
       ['DeltaView', 'False'],
       ['Buffer', 'False'],
       ['DH', 'False'],
       ['SIZE', 'False'],
       ['ByteArrayDataInputStream', 'False'],
       ['Rs

In [15]:
len(index)

379

In [16]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [17]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['DefaultSocketFactory', 'void', 'closeDatagramSocket', 'void', 'closeServerSocket', 'sock', 'throws', 'IOException', 'void', 'closeSocket', 'sock', 'throws', 'IOException', 'DatagramSocket', 'createDatagramSocketString', 'servicename', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameint', 'port', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameint', 'portInetAddress', 'laddr', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameSocketAddress', 'bindaddr', 'throws', 'SocketException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicename', 'throws', 'IOException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicenameint', 'port', 'throws', 'IOException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicenameSocketAddress', 'bindaddr', 't

In [18]:
# # 训练阶段
def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    # print(vectorizer.get_feature_names())# 获得模型直接分析数据找到的词汇量（上面单词的集合）
    # print(X.toarray())# 直接打印X输出的是每个词的位置
    #     print(X)

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  # 潜在语义分析，设置4个话题
    X2 = svd_model.fit_transform(X)  # 训练并进行转化

    # print("--------lsa奇异值---------")
    # print(svd_model.singular_values_)
    # print("--------文本在话题向量空间下的表示 左奇异矩阵---------")
    # print(X2)

    terms = vectorizer.get_feature_names()
    # print("--------话题及关键词---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [19]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['DefaultSocketFactory', 'void', 'closeDatagramSocket', 'void', 'closeServerSocket', 'sock', 'throws', 'IOException', 'void', 'closeSocket', 'sock', 'throws', 'IOException', 'DatagramSocket', 'createDatagramSocketString', 'servicename', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameint', 'port', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameint', 'portInetAddress', 'laddr', 'throws', 'SocketException', 'return', 'new', 'DatagramSocket', 'createDatagramSocketString', 'servicenameSocketAddress', 'bindaddr', 'throws', 'SocketException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicename', 'throws', 'IOException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicenameint', 'port', 'throws', 'IOException', 'return', 'new', 'MulticastSocket', 'createMulticastSocketString', 'servicenameSocketAddress', 'binda

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


return
int
value
thread
in
this
batches
 
************* ['AwaitInfo', 'AwaitInfoString', 'nameboolean', 'return', 'Returns', 'whether', 'is', 'boolean', 'return'] ****************
return
whether
boolean
returns
is
awaitinfo
awaitinfostring
 
************* ['Channel', 'ChannelJChannel', 'Returns', 'the', 'logical', 'name', 'of', 'a', 'given', 'member', 'The', 'lookup', 'is', 'from', 'the', 'local', 'cache', 'of', 'logical', 'address', 'logical', 'name', 'mappings', 'and', 'no', 'remote', 'communication', 'is', 'param', 'return', 'The', 'logical', 'name', 'for', 'String', 'getNameAddress', 'return', 'member', 'null', 'NameCachegetmember'] ****************
logical
the
cache
channeljchannel
local
from
namecachegetmember
 
************* ['Frag3Header', 'Frag3Headerint', 'idint', 'fragidint', 'Frag3Headerint', 'idint', 'fragidint', 'numfragsint', 'originallengthint'] ****************
fragidint
idint
numfragsint
frag3header
originallengthint
frag3headerint
 
************* ['GridOutputStream',

In [20]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

# c = list(zip(class_name,acsm))
# print(type(c))
c = np.array(c)
# print(c)
# print(c[0][0])

In [21]:
df =pd.read_csv('/content/drive/MyDrive/delesmell/dataset_class/JGroups-master/JGroups.csv') 

In [22]:
x= df.iloc[:, 0]

In [23]:
c

array([['DefaultSocketFactory', '0.311126253105148'],
       ['NAMING', '0.02939887646620688'],
       ['MergeId', '0.032434078363941486'],
       ['MsgStats', '0.6111112282041473'],
       ['AwaitInfo', '0.09876543209876554'],
       ['Channel', '0.04676015080859307'],
       ['Frag3Header', '0.16460905349794222'],
       ['GridOutputStream', '0.05252222705062518'],
       ['MethodLookup', '0.25'],
       ['PaddedAtomicLong', '0.25'],
       ['SocketFactory', '0.06769814827987565'],
       ['MFC', '0.07999999999999996'],
       ['NonReflectiveProbeHandler', '0.1093288134439432'],
       ['MethodInvoker', '0.17585493627007'],
       ['OneTimeAddressGenerator', '0.1250000000000002'],
       ['SiteAddress', '0.0594809450422609'],
       ['ExecutorEvent', '0.3200000000000002'],
       ['GridInputStream', '0.02925423928842205'],
       ['ExecutionServiceDemo', '0.16737792630904139'],
       ['FutureListener', '0.25'],
       ['MergeData', '0.21375739644970437'],
       ['Global', '0.075190

In [24]:
Brain_Class

array([['DefaultSocketFactory', 'False'],
       ['NAMING', 'False'],
       ['MergeId', 'False'],
       ['MsgStats', 'False'],
       ['AwaitInfo', 'False'],
       ['Channel', 'False'],
       ['Frag3Header', 'False'],
       ['GridOutputStream', 'False'],
       ['MethodLookup', 'False'],
       ['PaddedAtomicLong', 'False'],
       ['SocketFactory', 'False'],
       ['MFC', 'False'],
       ['NonReflectiveProbeHandler', 'False'],
       ['MethodInvoker', 'False'],
       ['OneTimeAddressGenerator', 'False'],
       ['SiteAddress', 'False'],
       ['ExecutorEvent', 'False'],
       ['GridInputStream', 'False'],
       ['ExecutionServiceDemo', 'False'],
       ['FutureListener', 'False'],
       ['MergeData', 'False'],
       ['Global', 'False'],
       ['RequestHandler', 'False'],
       ['DeliveryManager', 'False'],
       ['DeltaView', 'False'],
       ['Buffer', 'False'],
       ['DH', 'False'],
       ['SIZE', 'False'],
       ['ByteArrayDataInputStream', 'False'],
       ['Rs

In [25]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

*
*
*
*
*
*
*
*
*
*
*
*
*


In [26]:

ngList=[] 

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
            break
#             print('*')
    if q==1 :
        ngList.append(1)
        

In [27]:
brainList=[] 

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
            break
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

361
361


In [28]:
column=df.columns.tolist()

In [29]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [30]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,Counter,0.00,0,0,1.0,0,1,3,0.0,0,...,0,0,0,7,0,0.00,0,1.0,1,0
1,CounterService,1.17,0,0,1.0,0,1,3,0.0,0,...,2,0,0,6,0,1.00,7,1.0,1,0
2,Client,0.00,0,0,1.0,0,1,1,0.0,0,...,0,0,2,6,0,0.00,0,1.0,1,0
3,ConnectionListener,0.00,0,0,1.0,0,0,0,0.0,0,...,0,0,2,2,0,0.00,0,1.0,1,0
4,NioBaseServer,1.12,0,0,1.0,0,1,1,0.0,0,...,8,0,2,16,0,0.06,18,1.0,1,0
